In [2]:
import cv2
import time
import numpy as np
import pandas as pd
import mediapipe as mp
import plotly.express as px
import plotly.graph_objects as go

## YOLOv5 Setup

In [10]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
!pip install ipywidgets

Cloning into 'yolov5'...
remote: Enumerating objects: 10923, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10923 (delta 1), reused 5 (delta 1), pack-reused 10916
Receiving objects: 100% (10923/10923), 11.07 MiB | 33.74 MiB/s, done.
Resolving deltas: 100% (7540/7540), done.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 121 kB 25.1 MB/s 
     |████████████████████████████████| 243 kB 67.0 MB/s 
     |████████████████████████████████| 1.6 MB 76.3 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [11]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [69]:
# Model
# Select model version - https://github.com/ultralytics/yolov5
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)  # or yolov5m, yolov5l, yolov5x, custom
yolo.to(device)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-2-19 torch 1.10.2+cu102 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
           

## Pose + Object Detection

In [70]:
ANGLE_BGR_COLOR = (0,165,255)
POSE_BGR_COLOR = (255,0,0)
FPS_BGR_COLOR = (0,0,255)

In [71]:
# Class -> bounding box color
DETECTED_CLASSES = {
    'baseball bat': (0,255,0),
    'sports ball': (0,0,255)
}

BB_THICKNESS = 3

In [72]:
class poseDetector:
    def __init__(
        self,
        mode=False,
        complex=1,
        smooth_landmarks=True,
        segmentation=True,
        smooth_segmentation=True,
        detectionCon=0.5,
        trackCon=0.5,
    ):

        self.mode = mode
        self.complex = complex
        self.smooth_landmarks = smooth_landmarks
        self.segmentation = segmentation
        self.smooth_segmentation = smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpDrawStyle = mp.solutions.drawing_styles
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(
            self.mode,
            self.complex,
            self.smooth_landmarks,
            self.segmentation,
            self.smooth_segmentation,
            self.detectionCon,
            self.trackCon,
        )
        self.mp_drawing = mp.solutions.drawing_utils

    def findPose(self, img, draw = True):
        '''
        Pose estimation and drawing of the pose estimation to given frame.
        '''
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)    
        # self.plotly_fig(self.results.pose_landmarks)   
        # print(self.results.pose_landmarks)
          
        # Code to capture all body points and map it to a video
        if self.results.pose_landmarks:
            if draw:
                self.mp_drawing.draw_landmarks(
                    img,
                    self.results.pose_landmarks,
                    self.mpPose.POSE_CONNECTIONS,
                    self.mpDrawStyle.get_default_pose_landmarks_style())
                    # self.mpDraw.DrawingSpec(
                    #     color=(0, 0, 255), thickness=2, circle_radius=2
                    # ),
                    # self.mpDraw.DrawingSpec(
                    #     color=(0, 255, 0), thickness=2, circle_radius=2
                    # ),
        return img

    def findPosition(self, img, draw=False):
        '''
        Finds position of all the landmarks from previous pose estimation.
        '''
        self.lmList = []
        if self.results.pose_landmarks:
            # Save landmarks
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                x, y, z = lm.x, lm.y, lm.z
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        return self.lmList

    def findAngle(self, img, p1, p2, p3, draw=True):
        # Get the landmarks
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        # Calculate the Angle
        radians = np.arctan2(y3 - y2, x3 - x2) - np.arctan2(y1 - y2, x1 - x2)
        angle = radians * 180.0 / np.pi # np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = -(360 - angle)
        elif angle < -180:
            angle += 360

        # print(int(angle))

        # Draw
        if draw:
            # cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            # cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            # cv2.circle(img, (x1, y1), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x1, y1), 10, (0, 0, 255), 2)
            # cv2.circle(img, (x2, y2), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x2, y2), 10, (0, 0, 255), 2)
            # cv2.circle(img, (x3, y3), 5, (0, 0, 255), cv2.FILLED)
            # cv2.circle(img, (x3, y3), 10, (0, 0, 255), 2)
            cv2.putText(
                img,
                str(int(angle)) + "",
                (x2 - 50, y2 + 50),
                cv2.FONT_HERSHEY_PLAIN,
                2,
                ANGLE_BGR_COLOR,# (255, 0, 0),
                2,
            )
        return angle

In [73]:
def detect_objects(image):
    '''
    Detects objects and returns information for the bounding boxes.
    '''

    results = model(image)
    detected_df = results.pandas().xyxy[0]

    # Desired classes
    bounding_boxes = [] # (start_point, end_point, color)
    for class_name, bounding_box_color in DETECTED_CLASSES.items():
        # TODO: !!! Assume at most one ocurrence
        filtered_df = detected_df[detected_df['name'] == class_name]
        if len(filtered_df) > 0:
            detected_ocurrence = filtered_df.iloc[0].drop('name').astype(int)
            
            bb_start_point = (detected_ocurrence['xmin'], detected_ocurrence['ymin'])
            bb_end_point = (detected_ocurrence['xmax'], detected_ocurrence['ymax'])
            bounding_boxes.append((bb_start_point, bb_end_point, bounding_box_color))

    return bounding_boxes

In [74]:
OUTPUT_DIR = 'out'
VIDEO_OUT_PATH = 'out.mp4'
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [75]:
size = (1920, 1080)
fourcc = cv2.VideoWriter_fourcc(*'MP4V') # cv2.VideoWriter_fourcc(*'DIVX') 
FPS = 25
out = cv2.VideoWriter(VIDEO_OUT_PATH,fourcc,FPS, size)

In [76]:
def main():
    cap = cv2.VideoCapture('Hackathon_1st_Hitter.mp4')
    milliseconds = 1000
    # start_time = int(input("Enter Start time (in seconds): "))
    # end_time = int(input("Enter Length (in seconds): "))
    # end_time = start_time + end_time
    # cap.set(cv2.CAP_PROP_POS_MSEC, start_time * milliseconds)
    # pTime = 0
    detector = poseDetector()
    cnt = 0
    # while True and cap.get(cv2.CAP_PROP_POS_MSEC) <= end_time * milliseconds:
    success = True
    while success:
        success, img = cap.read()
        
        # Object detection
        detected_bounding_boxes = detect_objects(img)

        # Pose estimation + pose drawing
        img = detector.findPose(img, draw=True)
        lmList = detector.findPosition(img, draw=False)#, #draw=True)
        if len(lmList) != 0:
            detector.findAngle(img, 13, 11, 23)
            detector.findAngle(img, 24, 12, 14)

        # Drawing of object detection bounding boxes
        for bb_start_point, bb_end_point, bounding_box_color in detected_bounding_boxes:
            img = cv2.rectangle(img, bb_start_point, bb_end_point, bounding_box_color, BB_THICKNESS)

        # cTime = time.time()
        # fps = 1 / (cTime - pTime)
        # pTime = cTime

        # # show fps count
        # cv2.putText(
        #     img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3
        # )
        
        # Save output image
        # cv2.imwrite("{}/image_{}.jpg".format(OUTPUT_DIR, cnt), img)
        
        # Output frame to the video
        out.write(img)

        cnt += 1

    out.release()

In [77]:
main()

AttributeError: 'NoneType' object has no attribute 'shape'

In [53]:
import multiprocessing

multiprocessing.cpu_count()

# TODO - multithread processing for Mediapipe Pose and batch processing for YOLOv5

4

In [78]:
out.release()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d9b3f91b-9816-4bae-8004-e94b885975fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>